# Reporte diario promesas realizadas durante el dia

In [28]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import date
from datetime import datetime
import ConeccionesSucursales as cs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [29]:
#Definimos la ruta en donde queremos guardar los archivos
mes2 = 'Enero2021'
ruta = '/home/estadistico/Documents/Erick/'+mes2+'/Reportes diarios/Promesas diarias'
ruta_pagos = '/home/estadistico/Documents/Erick'
#Definimos la ruta donde estan las asignaciones
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'

In [30]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = 3306
usuario = 'estadisticas'
contraseña = 'estadisticas8474'
base = 'procesos_externos'

In [31]:
#Asignamos valores a los parametros \n",
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y-%H-%M')
today

'20210111'

In [32]:
#Hacemos la conexion con el servidor\n",
try:
    conn = mysql.connector.connect(user=usuario,
                               password=contraseña,
                               host=servidor,
                               port=puerto,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

# Liverpool

In [33]:
#Hacemos la consulta referente a las gestiones de Liverpool
gestion_liv = cs.query_naucalpan('liverpool_rpt_gestiones_detallado',today,today)
gestion_liv.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv = gestion_liv.loc[gestion_liv['dictamen']=='PROMESA']

In [34]:
#Asignacion Liverpool
asignacion_liv = pd.read_csv(''+ruta_pagos+'/Liverpool/Asignacion/Asignaciones csv/Asignacion '+mes2+'.csv',converters={'# CUENTA':str})
asignacion_liv = asignacion_liv[['ID AGENCIA', '# CUENTA']]
asignacion_liv.columns = ['division','credito']
asignacion_liv.head(1)

,division,credito
0,SA002REM,0001400000007731973


In [35]:
#Hacemos el consolidado 
consolidado_liv = pd.merge(gestion_liv,asignacion_liv,how='left',on=['credito'])
cols = ['monto_promesa','credito']
consolidado_liv[cols] = consolidado_liv[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv = consolidado_liv[(consolidado_liv['monto_promesa']>50)]
consolidado_liv = consolidado_liv[(consolidado_liv['monto_promesa']<500000)]
consolidado_liv.fillna('COBRANZA',inplace=True)
consolidado_liv.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,fecha_promesa,monto_promesa,comentarios,division
1,9594579,12,2,1400000001083306,MARTIN PATLAN COTE,5533942597,celular,2021-01-11 15:29:24,YHERNANDEZBA,ALEJANDRA YESENIA HERNANDEZ BITE,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2021-01-15,1000.0,5533942597 SE REPORTA TT PARA AGENDAR CONVENIO...,SA001REM
2,9594364,12,3,4050670000377912,ALEJANDRO CUANCA NAJERA,7713834418,casa,2021-01-11 15:10:50,YHERNANDEZBA,ALEJANDRA YESENIA HERNANDEZ BITE,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2021-01-11,3337.0,7713834418 MENCIONA QUE EL DIA 11/01/21 LE LLE...,SA002REM
3,9594304,12,4,13000024252145,GABRIEL OLVERA VAZQUEZ,4423616851,celular,2021-01-11 14:48:44,BARRETOOSMF,BARRETO OSEGUERA MARIA FERNANDA,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2021-01-15,2250.0,"4423616851 // PROMESA DE PAGO PARCIAL , TT LIQ...",DKN00REM


In [36]:
#Obtenemos datos de las promesas de Liverpool
pivot_liv = pd.pivot_table(consolidado_liv,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_liv = pd.DataFrame(pivot_liv.to_records())
pivot_liv = pivot_liv.iloc[:,[0,1,4]]
pivot_liv.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_liv.fillna(0,inplace=True)
#pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_liv

,Division,NumeroPromesas,SumaPromesas
0,DKN00REM,8,21003.0
1,DKN02REM,8,17418.0
2,DKN04REM,13,28222.0
3,SA001REM,11,21429.0
4,SA002REM,2,4327.0
5,All,42,92399.0


In [37]:
#Vemos las promesas reales eliminando duplicados
consolidado_liv2 = consolidado_liv.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_liv2 = pd.pivot_table(consolidado_liv2,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_liv2 = pd.DataFrame(pivot_liv2.to_records())
pivot_liv2 = pivot_liv2.iloc[:,[0,1,4]]
pivot_liv2.columns = ['Dictamen','NumeroPromesas','SumaPromesas']
pivot_liv2.fillna(0,inplace=True)
#pivot_liv2[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_liv2[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)


# Bradesco

In [38]:
#Hacemos la consulta referente a las gestiones de Bradesco
gestion_brad = cs.query_naucalpan('bradescard_rpt_gestiones_detallado',today,today)
gestion_brad.columns = ['folio_gestion','firma_id','unegocio_id','NUMERO_TARJETA','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad = gestion_brad.loc[gestion_brad['dictamen']=='PROMESA']
gestion_brad['NUMERO_TARJETA'] = pd.to_numeric(gestion_brad['NUMERO_TARJETA']) 

In [39]:
asignacion_brad_c0 = pd.read_excel(''+ruta_pagos+'/Bradesco/Asignacion_c0/ASIGNACION C0 '+mes2+'.xlsx')
asignacion_brad_c0 = asignacion_brad_c0.iloc[:,[0,2,3,8,10,11,14,25]]
asignacion_brad_extra = pd.read_csv(''+ruta_pagos+'/Bradesco/Asignacion_extra/CSV/Asignacion Extra '+mes2+'.csv',encoding= 'unicode_escape')
asignacion_brad_extra = asignacion_brad_extra.iloc[:,[0,2,3,5,9,25,26,28]]
asignacion_brad_extra.columns = ['CODIGO_ORG','NUMERO_CUENTA','NUMERO_TARJETA','RFC','ESTADO','SALDO_TOTAL','PAGO_DESCUENTO','PAGOSVENCIDOS']

asignacion_brad = pd.concat([asignacion_brad_c0,asignacion_brad_extra],ignore_index=True)

In [40]:
#Hacemos la union de asignacion y promesas
consolidado_brad = pd.merge(gestion_brad,asignacion_brad,how='left',on=['NUMERO_TARJETA'])
cols = ['monto_promesa']
consolidado_brad[cols] = consolidado_brad[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad = consolidado_brad[(consolidado_brad['monto_promesa']>50)]
consolidado_brad = consolidado_brad[(consolidado_brad['monto_promesa']<500000)]

In [41]:
#Obtenemos datos de las promesas de Bradesco
pivot_brad = pd.pivot_table(consolidado_brad,index=['PAGOSVENCIDOS'],values=['NUMERO_TARJETA','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_brad = pd.DataFrame(pivot_brad.to_records())
pivot_brad = pivot_brad.iloc[:,[0,1,4]]
pivot_brad.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_brad.fillna(0,inplace=True)
#pivot_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_brad

,Division,NumeroPromesas,SumaPromesas
0,A,1,2000.0
1,C0 4PV,19,33440.0
2,C0 5PV,14,22908.0
3,C0 6PV,15,32130.0
4,C1,18,26610.0
5,C2,12,19566.0
6,C3,11,12869.0
7,C4,3,5050.0
8,All,93,154573.0


In [42]:
#Vemos las promesas reales eliminando duplicados
consolidado_brad2 = consolidado_brad.drop_duplicates(subset=['NUMERO_TARJETA','fecha_promesa','monto_promesa'])
pivot_brad2 = pd.pivot_table(consolidado_brad2,index=['PAGOSVENCIDOS'],values=['NUMERO_TARJETA','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_brad2 = pd.DataFrame(pivot_brad2.to_records())
pivot_brad2 = pivot_brad2.iloc[:,[0,1,4]]
pivot_brad2.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_brad2.fillna(0,inplace=True)
#pivot_brad2[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad2[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_brad2

,Division,NumeroPromesas,SumaPromesas
0,A,1,2000.0
1,C0 4PV,19,33440.0
2,C0 5PV,14,22908.0
3,C0 6PV,15,32130.0
4,C1,18,26610.0
5,C2,12,19566.0
6,C3,11,12869.0
7,C4,3,5050.0
8,All,93,154573.0


# Credifiel

In [43]:
#Hacemos la consulta referente a las gestiones de Credifiel
gestion_cred = cs.query_naucalpan('credifiel_rpt_gestiones_detallado',today,today)
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']
gestion_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
cols = ['monto_promesa','unegocio_id']
gestion_cred[cols] = gestion_cred[cols].apply(pd.to_numeric, errors='coerce')
gestion_cred = gestion_cred.loc[gestion_cred['monto_promesa']>0]

In [44]:
#Hacemos la union de asignacion y promesas
consolidado_cred = gestion_cred
cols = ['monto_promesa','unegocio_id']
consolidado_cred[cols] = consolidado_cred[cols].apply(pd.to_numeric, errors='coerce')
consolidado_cred = consolidado_cred[(consolidado_cred['monto_promesa']>50)]
consolidado_cred = consolidado_cred[(consolidado_cred['monto_promesa']<500000)]
consolidado_cred['division'] = 'EXTRAJUDICIAL'

In [45]:
#Obtenemos datos de las promesas de Credifiel
pivot_cred = pd.pivot_table(consolidado_cred,index=['division'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_cred = pd.DataFrame(pivot_cred.to_records())
pivot_cred = pivot_cred.iloc[:,[0,3,2]]
pivot_cred.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_cred.fillna(0,inplace=True)
pivot_cred = pivot_cred.loc[pivot_cred['SumaPromesas']>0]
#pivot_gestion_cred[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_cred

,Division,NumeroPromesas,SumaPromesas
0,EXTRAJUDICIAL,3,4669.0


In [46]:
#Vemos las promesas reales eliminando duplicados
consolidado_cred2 = consolidado_cred.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_cred2 = pd.pivot_table(consolidado_cred2,index=['division'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_cred2 = pd.DataFrame(pivot_cred2.to_records())
pivot_cred2 = pivot_cred2.iloc[:,[0,3,2]]
pivot_cred2.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_cred2.fillna(0,inplace=True)
pivot_cred2 = pivot_cred2.loc[pivot_cred2['SumaPromesas']>0]
#pivot_gestion_cred[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_cred2

,Division,NumeroPromesas,SumaPromesas
0,EXTRAJUDICIAL,3,4669.0


# Banco Azteca

In [47]:
#Hacemos la consulta referente a las gestiones de Banco Azteca
gestion_baz = cs.query_naucalpan('baz_rpt_gestiones_detallado',today,today)
gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']

In [48]:
#new_asignaciones = os.listdir('/home/estadistico/Documents/Erick/Banco Azteca/Asignacion')
#sample_cols_to_keep =['#FIIDCAMPANA', 'FIPAIS', 'FICANAL','FISUCURSAL','FIFOLIO','FISEMATRASMAX','FNSALDO','FNSALDOCAPITAL','FNPAGOREQ','FCTEL1','FCTIPO1','FNDIA_PAGO','FIDIASATRASOMAX','FNCAPPAGODISP','FNABONOSEMANAL','FNCAPACIDADPAGO','FILCRACTIVA','FDFECPROXPAG','FNTASAINT','CP','fcnombreos','fcappaternoos','FITERRITORIO','FCDESCTERRITORIO','FIZONA','FCDESCZONA','FIREGION','FCDESCREGION','FIGERENCIA','FCDESCGERENCIA','FCBESTTIMETOCALL','FECHA']
#df_iter = pd.read_csv('/home/estadistico/Documents/Erick/Banco Azteca/Asignacion_csv/Asignacion banco azteca '+mes+'.csv', chunksize=20000, usecols=sample_cols_to_keep) 
#df_lst = [] 
#for df_ in df_iter: 
#            tmp_df = (df_.rename(columns={col: col.lower() for col in df_.columns}))
#            df_lst += [tmp_df.copy()] 
#asignacion_baz = pd.concat(df_lst)

In [49]:
#asignacion_baz = pd.read_csv(''+ruta_asig_baz+'/Asignacion banco azteca '+str(mes2)+'.csv')
#asignacion_baz = asignacion_baz.rename(columns=str.lower)
#asignacion_baz.head(3)

In [50]:
#asignacion_baz['credito'] = asignacion_baz['fipais'].map(str)+'-'+asignacion_baz['ficanal'].map(str)+'-'+asignacion_baz['fisucursal'].map(str)+'-'+asignacion_baz['fifolio'].map(str)
#asignacion_baz.head(3)

In [51]:
consolidado_baz = gestion_baz
cols = ['monto_promesa']
consolidado_baz[cols] = consolidado_baz[cols].apply(pd.to_numeric, errors='coerce')
consolidado_baz = consolidado_baz[(consolidado_baz['monto_promesa']>50)]
consolidado_baz = consolidado_baz[(consolidado_baz['monto_promesa']<500000)]
consolidado_baz.fillna('COBRANZA',inplace=True)

In [52]:
#Obtenemos datos de las promesas de Banco Azteca
pivot_baz = pd.pivot_table(consolidado_baz,index=['dictamen'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_baz = pd.DataFrame(pivot_baz.to_records())
pivot_baz = pivot_baz.iloc[:,[0,3,2]]
pivot_baz.columns = ['Dictamen','NumeroPromesas','SumaPromesas']
pivot_baz.fillna(0,inplace=True)
#pivot_baz[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_baz[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_baz

,Dictamen,NumeroPromesas,SumaPromesas
0,PROMESA,98,96612.0


In [53]:
#Vemos las promesas reales eliminando duplicados
consolidado_baz2 = consolidado_baz.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_baz2 = pd.pivot_table(consolidado_baz2,index=['dictamen'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_baz2 = pd.DataFrame(pivot_baz2.to_records())
pivot_baz2 = pivot_baz2.iloc[:,[0,3,2]]
pivot_baz2.columns = ['Dictamen','NumeroPromesas','SumaPromesas']
pivot_baz2.fillna(0,inplace=True)
#pivot_baz2[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_baz2[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_baz2

,Dictamen,NumeroPromesas,SumaPromesas
0,PROMESA,98,96612.0


In [55]:
writer = pd.ExcelWriter(''+ruta+'/Reporte Promesas Diarias '+now+'.xlsx', engine='xlsxwriter')
pivot_liv.to_excel(writer,'PromDiariaLivBruto',index=False,header=True)
pivot_liv2.to_excel(writer,'PromDiariaLivReal',index=False,header=True)
pivot_brad.to_excel(writer,'PromDiariaBradBruto',index=False,header=True)
pivot_brad2.to_excel(writer,'PromDiariaBradReal',index=False,header=True)
pivot_cred.to_excel(writer,'PromDiariaCredBruto',index=False,header=True)
pivot_cred2.to_excel(writer,'PromDiariaCredReal',index=False,header=True)
pivot_baz.to_excel(writer,'PromDiariaBAZBruto',index=False,header=True)
pivot_baz2.to_excel(writer,'PromDiariaBAZReal',index=False,header=True)

writer.save()
# writer.close()